# Week 04a Assignment weather data

Welcome to week four of this course programming 1. Analyzing time related data such as estimating seasonal effect, or year effect might be a challenge. How to filter the essential information from the noise? How to apply signal analysis with noisy data. How to make compact useful visualizations? Python has several constructs to handle date time related data. The relevant classes for making plots are Locators and Formatters. Locators determine where the ticks are, and formatters control the formatting of tick labels. The relevant class for date time data is the pandas datetime data type, which has methods like resample and several possibilities to display data (frequencies). As a study case we will work with weather data. If you have data that fits the learning goals, you can bring your own data.

Keywords: signal processing, smoothing, resample, formatters and locators, datetime object

More to read: 

- https://fennaf.gitbook.io/bfvm22prog1/
- https://machinelearningmastery.com/time-series-data-visualization-with-python/
- https://towardsdatascience.com/how-to-plot-time-series-86b5358197d6
- In the https://pandas.pydata.org/docs/reference/offset_frequency.html you can find more about frequencies and in the documentation
- https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html you can read all the methods of this datetime object.
- https://en.wikipedia.org/wiki/Smoothing


Learning objectives

- load, inspect and clean a dataset
- reshape dataframes to group data in a certain frequency
- apply smoothing technologies
- Create useful visualisation with timeseries data
- Maintain development environment 
- Apply coding standards and FAIR principles

Please add topics you want to learn here: https://padlet.com/ffeenstra1/z9duo25d39dcgezz


## Assignment

You will to organise your data into the required format and apply smoothing. In this assignment we will work with weatherdata from the KNMI. A subset of weatherdata is for you available in the file: `KNMI_20181231`. The data consist of several stations with daily weather data of several years. Your task is to make a plot similar to the plot below. 

<img src="../images/weather.png" alt="drawing" width="400"/>


Furthermore the plot needs the following enhancements

1. proper titles and ticks
2. widgets selecting a particular year or all years
3. lines need to be smoothed
3. legends needs to be added

Use your creativity. Consider colors, alpha settings, sizes etc. 

Learning outcomes

- load, inspect and clean a dataset 
- reformat dataframes
- apply smoothing technologies
- visualize timeseries data

The assignment consists of 6 parts:

- [part 1: load the data](#0)
- [part 2: clean the data](#1)
- [part 3: reformat data](#2)
- [part 4: smooth the data](#3)
- [part 5: visualize the data](#4)
- [part 6: Challenge](#5)

Part 1 and 5 are mandatory, part 6 is optional (bonus)
Mind you that you cannot copy code without referencing the code. If you copy code you need to be able to explain your code verbally and you will not get the full score. 


NB if you want to make a plot with more actual data you can download data from https://openweathermap.org/api 


---

<a name='0'></a>
## Part 1: Load the data

Either load the dataset `KNMI_20181231.csv` or `KNMI_20181231.txt.tsv`. 
Preferably we read the data not with a hard coded data path but using a config file. See https://fennaf.gitbook.io/bfvm22prog1/data-processing/configuration-files/yaml. The dataheaders contain spaces and are not very self explainable. Change this into more readable ones. Select data from a station. Station 270 is in the neighborhood of Groningen. For our plot we only need the the mean, minimum and maximum temperature. Of course you are welcome to select other data if you think it might be useful for your visualization. The data should look something like this:


In [1]:
import pandas as pd
import numpy as np
import panel as pn
import os
import re
import yaml
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.models import Band, ColumnDataSource, DatetimeTickFormatter
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt 


In [2]:
def configReader():
    """
    explanation: This function open config,yaml file 
    and fetch the gonfigue file information
    input: ...
    output: configue file
    """
    with open("config.yaml", "r") as inputFile:
        config = yaml.safe_load(inputFile)
    return config

In [3]:
#read the file
config = configReader()
FileName = config.values()
weatherDf = pd.read_table(list(FileName)[0], skiprows=64)

#make a dataframe from the file and add some feature to it
weatherDf = weatherDf['#'].str.split(",", expand = True )
weatherDf = weatherDf.drop(weatherDf.columns[5:], axis='columns')

weatherDf.columns = ['station', 'Date', 'Tmean','Tmin', 'Tmax']
weatherDf['Date'] = weatherDf['Date'].astype('M')

#seperate Leeuwarden weather information from the main data
weatherDfLeeuwarden = weatherDf[weatherDf['station'].astype('i') == 270]
weatherDfLeeuwarden.head()

,station,Date,Tmean,Tmin,Tmax
97641,270,2000-01-01,42,-4,79
97642,270,2000-01-02,55,33,74
97643,270,2000-01-03,74,49,89
97644,270,2000-01-04,46,22,75
97645,270,2000-01-05,41,14,56


---

<a name='1'></a>
## Part 2: Clean the data

The data ia not clean. There are empty cells in the dataframe which needs to be replaced with NaN's and the temperature is in centidegrees which needs to be transformed into degrees. The date field needs a datetime format. For visualization convience we would like to remove the leap year. Conduct the cleaning.

In [4]:
#replace cells with spaces to NaN
weatherDf = weatherDf.replace("     ", np.NaN)

#change data formats
weatherDf.iloc[:,2:] = weatherDf.iloc[:,2:].astype(np.dtype("float64"))
weatherDf.station = weatherDf['station'].astype(np.dtype("int64"))

#change temperatures to celcius degrees
weatherDf.iloc[:,2:] = (weatherDf.iloc[:,2:] * 0.1).round(decimals=2)
#remove leap year
weatherDf = weatherDf[~((weatherDf.Date.dt.month == 2) & (weatherDf.Date.dt.day == 29))]
weatherDf = weatherDf.set_index(['Date'])

#seperate Leeuwarden weather information from the main data
weatherDfLeeuwarden = weatherDf[weatherDf['station'].astype('i') == 270]



<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<ul><li>pd.to_datetime(df['Date'].astype(str), format='%Y%m%d')</li>
    <li>regex for empty cells = `^\s*$` </li>
    <li>remove month == 2 & day == 29</li> 
</ul>
</details>

In [5]:
#Test your outcome
#write code to check if you have done the above
try:
    weatherDfLeeuwarden.info()
except:
    print("weatherDf not properly loaded")

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 6935 entries, 2000-01-01 to 2018-12-31
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   station  6935 non-null   int64  
 1   Tmean    6935 non-null   float64
 2   Tmin     6935 non-null   float64
 3   Tmax     6935 non-null   float64
dtypes: float64(3), int64(1)
memory usage: 270.9 KB


### Expected outcome

---

<a name='2'></a>
## Part 3: Reform your data

First we will split the data in data from 2018 and data before 2018. Best is to split this in two dataframes. 
Next we need for the non 2018 data the minimum values for each day and the maximum values for each day. So we look for the minimum value out of all january-01 minimum values (regardless the year). Create a dataframe with 365 days containing the ultimate minimum and the ultimate maximum per day. 


In [6]:
def month_day(df_multipleyears):
    """
    input: data frame
    explanation: this function makes group based on month and day 
    and find the total extermums
    output:minimum data frame, maximum data frame
    """
    minDf = pd.DataFrame(df_multipleyears.groupby([df_multipleyears.index.month,
                                 df_multipleyears.index.day]).min())
    
    maxDf = pd.DataFrame(df_multipleyears.groupby([df_multipleyears.index.month, 
                                 df_multipleyears.index.day]).max()) 
    return minDf, maxDf

In [7]:

def test_reformed(df):
    #Test the code
    df = df[(df.index.year > 2007) & (df.index.year < 2018)]   
    return month_day(df)

   


In [8]:
after2018Df =  weatherDfLeeuwarden[weatherDfLeeuwarden.index.year == 2018]
before2018Df = weatherDfLeeuwarden [weatherDfLeeuwarden.index.year != 2018]
minBefore2018, maxBefore2018 = test_reformed(before2018Df)
minBefore2018.Tmin

Date  Date
1     1       -5.8
      2       -7.5
      3      -12.6
      4       -4.1
      5       -6.0
              ... 
12    27      -4.8
      28      -4.2
      29      -6.7
      30     -10.2
      31     -10.6
Name: Tmin, Length: 365, dtype: float64

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<ul><li>use the dt.month and dt.day to groupby</li>
</ul>
</details>

### Expected outcome
Note, the layout or names my differ, but the length should be 365 and the minimum values should be the same

---

<a name='3'></a>
## Part 4: Smooth the data

Make a function that takes an array or a dataframe column and returns an array of smoothed data. Explain in words why you choose a certain smoothing algoritm. Ask the signal analysis teacher if you want some advice.


In [9]:

def data_smoother(df):

    #calculate rolling mean
    rollingMean = df.rolling(window=10).mean()
    avg = df.loc[1].loc[:10].mean()
    rollingMean.fillna(avg, inplace=True)

    #calculate exponential smoothing technique
    expSmoothing = SimpleExpSmoothing(df).fit(smoothing_level=0.2, optimized=False)

    #calculate halt smoothing technique
    holtSmoothing =Holt(df).fit()

    return  expSmoothing, rollingMean, holtSmoothing
    
smoothMinBefore2018 = data_smoother(minBefore2018.Tmin)


c:\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [10]:
#plot different smoothed data and the main data to compare which on 
#can preent the data more properly.
fig = figure(title=f'Smooth line', plot_width=1000, plot_height=600, 
             x_axis_label='x', y_axis_label='y')

fig.line(x=range(0,len(minBefore2018.Tmin)), y=minBefore2018.Tmin, 
         legend_label= 'main line', line_width=1, color='red')

fig.line(x=range(0,len(minBefore2018.Tmin)), y=smoothMinBefore2018[0].fittedvalues, 
         legend_label= 'exp. smooth line', line_width=1, color='blue')

fig.line(x=range(0,len(minBefore2018.Tmin)), y=smoothMinBefore2018[1], 
         legend_label= 'rolling mean line', line_width=1, color='green')

fig.line(x=range(0,len(minBefore2018.Tmin)), y=smoothMinBefore2018[2].fittedvalues, 
         legend_label= 'halt smooth line', line_width=1, color='black')

output_notebook()
show(fig)

Loading BokehJS ...

#your motivation here
These are all possible smoothing techniques. but the one that I think is the best one in this question is the exponential technique. Because it smooth the data properly in comparison with other techniques. we cannot use exponential halt technique here due to the existance of zero in the denumerator, also linear halt cannot smooth the data in a proper way. in order to get a proper reault from rolling mean, we should make window equal to 10 which means we loose ten first data. so the most proper one is exponential technique.

---

<a name='4'></a>
## Part 5: Visualize the data

Plot the mean temperature of the year 2018. Create a shaded band with the ultimate minimum values and the ultimate maximum values from the multi-year dataset. Add labels, titles and legends. Use proper ranges. Be creative to make the plot attractive. 



<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<ul><li>use from bokeh.models import Band</li>
    <li>use ColumnDataSource to parse data arrays</li>
    <li>look for xaxis tick formatters</li>
</ul>
</details>

---

In [11]:

source = ColumnDataSource(data=after2018Df.reset_index())
p = figure(title='temprature (2018)', x_axis_type='datetime', x_axis_label='time',
           y_axis_label='temprature',plot_width=1000, plot_height=600)

p.line(x='Date', y='Tmean', legend_label= 'Tmean', line_width=1, color='blue',
       source=source)

band = Band(base='Date', lower='Tmin', upper='Tmax', source=source, level='underlay',
            fill_alpha=1.0, line_width=1, line_color='black')

p.add_layout(band)

output_notebook()
show(p)


Loading BokehJS ...

<a name='5'></a>
## Part 6: Challenge

Make a widget in which you can select the year range for the multiyear set. Or maybe a widget were you choose a different station. Add this to your layout to make the plot interactive. Add another widget to select or deselect the smoother. Inspiration: https://demo.bokeh.org/weather

In [12]:
pn.extension()

In [13]:
def renderPlot(station=270, year=2018, smoother='rolling mean'):
    """
    Input= station number, year 
    Explanation= give a station number and year and plot the shaddy graph
    Output= graph
    """

    df = weatherDf[weatherDf['station'] == station]
    df = df.reset_index().dropna()
    df = df[df.Date.dt.year == year]
    source = ColumnDataSource(data=df)
    
    p = figure(title='temprature', x_axis_type='datetime', x_axis_label='time', 
                     y_axis_label='temprature',plot_width=1000, plot_height=600) 
    
    p.line(x='Date', y='Tmean', legend_label= 'Tmean', line_width=1, color='blue',source=source)
    
    band = Band(base='Date', lower='Tmin', upper='Tmax', source=source, level='underlay',
            fill_alpha=1.0, line_width=1, line_color='black')
    p.xaxis[0].formatter = DatetimeTickFormatter()  
    p.add_layout(band)

    p.line(x=df['Date'], y=smoothLineMaker(df.Tmean, smoother), 
         legend_label= smoother, line_width=1, color='red')
    
    return(p)


In [14]:
def smoothLineMaker(df, smoother):

    """
    Input= data frame (avarage tempreture series), smoothing technique
    Explanation= recieve a series and a smoothing type and make a series for plot
    Output= series for plot
    """    
    
    if smoother == 'rolling mean':
        smoothDf = df.rolling(window=10).mean()
        return smoothDf

    elif smoother == 'exponential':
        smoothDf = SimpleExpSmoothing(df).fit(smoothing_level=0.2, optimized=False)
        return smoothDf.fittedvalues
    
    elif smoother == 'halt':
        smoothDf =Holt(df).fit()
        return smoothDf.fittedvalues
    

In [15]:
#make the graph interactive
stationSeries = weatherDf['station']
newDf = weatherDf.reset_index()
yearSeries = newDf.Date.dt.year
smoother = ['exponential', 'rolling mean', 'halt']
interPlot = pn.interact(renderPlot, station=stationSeries, year=yearSeries, smoother=smoother)


In [16]:
#make a dashboard for it
dashboard = pn.template.BootstrapTemplate(title='weather cast',sidebar_width=200)
dashboard.sidebar.append(interPlot[0])
dashboard.main.append(interPlot[1])
dashboard.show()



Launching server at http://localhost:58526
